<a href="https://colab.research.google.com/github/BhavyaBansal49/NBA-Draft-Pick-Optimizer/blob/main/DFS_Fantasy_Picks_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DFS Fantast Picks Optimization Model Class Project


## Preliminary Data Processes

*   Import pyomos
*   Import other libraries like numpy and pandas
*   Mount Google Drive

### Import Pyomos

In [1]:
%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

     |████████████████████████████████| 9.1 MB 3.6 MB/s 
     |████████████████████████████████| 49 kB 3.7 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 148492 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unp

### Importing pandas, numpy

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

### Mounting Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Data Set Up

*   Read the daily fantasy fuel data file
*   Delete the unnecessary columns
*   A note about players listed as 'O'
*   A note about players listed as 'Q'
*   Adding columns to indicate positions available for a player
*   Adding a column to reflect the game player is participating in
*   Making necessary daa for the model

### Reading the file


In [4]:
my_fantasy_fuel_data = pd.read_csv("/content/drive/MyDrive/Project/NBA Draft Optimizer/NBA Data.csv")
my_fantasy_fuel_data

,first_name,last_name,position,position_alt,injury_status,game_date,slate,team,opp,spread,over_under,implied_team_score,salary,L5_ppg_floor,L5_ppg_avg,L5_ppg_max,ppg_projection,value_projection,ppg_actual,value_actual
0,Luka,Doncic,PG,SF,NaN,2021-05-28,NaN,DAL,LAC,2.0,219.5,108.8,11000,10,37.5,55.9,70.0,53.8,4.89,NaN
1,James,Harden,PG,SG,NaN,2021-05-28,NaN,BKN,BOS,-7.5,227.0,117.3,10300,15,5.8,36.0,53.8,49.0,4.76,NaN
2,Kevin,Durant,SF,PF,NaN,2021-05-28,NaN,BKN,BOS,-7.5,227.0,117.3,9600,21,30.3,45.5,53.5,46.5,4.84,NaN
3,Julius,Randle,PF,C,NaN,2021-05-28,NaN,NY,ATL,4.0,211.0,103.5,9700,28,37.0,48.0,71.3,46.3,4.77,NaN
4,Jayson,Tatum,PF,NaN,NaN,2021-05-28,NaN,BOS,BKN,7.5,227.0,109.8,9400,14,22.5,45.4,73.5,45.0,4.79,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Brandon,Goodwin,SG,NaN,O,2021-05-28,NaN,ATL,NY,-4.0,211.0,107.5,3000,2,6.5,10.8,18.8,0.0,0.00,NaN
73,Jaylen,Brown,SG,NaN,O,2021-05-28,NaN,BOS,BKN,7.5,227.0,109.8,3000,17,34.3,46.4,67.3,0.0,0.00,NaN
74,JJ,Redick,SG,NaN,O,2021-05-28,NaN,DAL,LAC,2.0,219.5,108.8,3000,12,0.0,4.6,9.5,0.0,0.00,NaN
75,Spencer,Dinwiddie,SG,NaN,O,2021-05-28,NaN,BKN,BOS,-7.5,227.0,117.3,3000,20,15.0,28.7,39.0,0.0,0.00,NaN


### Deleting Unecessary Columns

In [5]:
my_fantasy_fuel_data =  my_fantasy_fuel_data.drop(['game_date','slate','spread','over_under','implied_team_score','L5_ppg_floor','L5_ppg_avg','L5_ppg_max','ppg_projection','value_actual'], axis = 1)
my_fantasy_fuel_data

,first_name,last_name,position,position_alt,injury_status,team,opp,salary,value_projection,ppg_actual
0,Luka,Doncic,PG,SF,NaN,DAL,LAC,11000,53.8,4.89
1,James,Harden,PG,SG,NaN,BKN,BOS,10300,49.0,4.76
2,Kevin,Durant,SF,PF,NaN,BKN,BOS,9600,46.5,4.84
3,Julius,Randle,PF,C,NaN,NY,ATL,9700,46.3,4.77
4,Jayson,Tatum,PF,NaN,NaN,BOS,BKN,9400,45.0,4.79
...,...,...,...,...,...,...,...,...,...,...
72,Brandon,Goodwin,SG,NaN,O,ATL,NY,3000,0.0,0.00
73,Jaylen,Brown,SG,NaN,O,BOS,BKN,3000,0.0,0.00
74,JJ,Redick,SG,NaN,O,DAL,LAC,3000,0.0,0.00
75,Spencer,Dinwiddie,SG,NaN,O,BKN,BOS,3000,0.0,0.00


### Modifying the value projections for players who are marked as 'O'

These players already have a value projection of 0, therefore we should keep them in the data as a cheaper player who might be marked as 'O' can help to create a lineup with a higher overall draft points projection.

Have commented the code for the same reason.

In [ ]:
# player_out = my_fantasy_fuel_data['injury_status']
# for i in player_out:
#   if i == 'O':
#     my_fantasy_fuel_data['value_projection'] = my_fantasy_fuel_data['value_projection'] * 0
#   else:
#     my_fantasy_fuel_data['value_projection'] = my_fantasy_fuel_data['value_projection']

### Modifying the value projections for players who are marked as 'Q'
Initially, we planned to multiply the value projection of players marked as questionable with 0.8 and see if they would still be selected for the game or not.

However, we are finally going ahead with making a game time decision and if needed, force removing that certain player from the team.

Code has been commented out for the same.

In [ ]:
# questionable = my_fantasy_fuel_data['injury_status']
# for i in questionable:
#   if i == 'Q':
#     my_fantasy_fuel_data['value_projection'] = my_fantasy_fuel_data['value_projection'] * 0.8
#   else:
#     my_fantasy_fuel_data['value_projection'] = my_fantasy_fuel_data['value_projection']

### Adding columns to indicate positions available

In [6]:
## Adding columns in the initial dataframe to reflect positions that are available for a player

## If the rightmost letter of position or position_alt is G, then the person qualifies for being a guard
my_fantasy_fuel_data['Guard?']  = np.where((my_fantasy_fuel_data['position'].str.strip().str[-1] == 'G') | (my_fantasy_fuel_data['position_alt'].str.strip().str[-1] == 'G'),1,0)

## If the rightmost letter of position or position_alt is F, then the person qualifies for being a forward
my_fantasy_fuel_data['Forward?']  = np.where((my_fantasy_fuel_data['position'].str.strip().str[-1] == 'F') | (my_fantasy_fuel_data['position_alt'].str.strip().str[-1] == 'F'),1,0)

## Everyone can be played at UTIL
my_fantasy_fuel_data['UTIL?']  = 1

## If the rightmost letter of position or position_alt is PG, then the person qualifies for being a point guard
my_fantasy_fuel_data['PG?']  = np.where((my_fantasy_fuel_data['position'] == 'PG') | (my_fantasy_fuel_data['position_alt'] == 'PG'),1,0)

## If the rightmost letter of position or position_alt is SG, then the person qualifies for being a shooting guard
my_fantasy_fuel_data['SG?']  = np.where((my_fantasy_fuel_data['position'] == 'SG') | (my_fantasy_fuel_data['position_alt'] == 'SG'),1,0)

## If the rightmost letter of position or position_alt is SF, then the person qualifies for being a small forward
my_fantasy_fuel_data['SF?']  = np.where((my_fantasy_fuel_data['position'] == 'SF') | (my_fantasy_fuel_data['position_alt'] == 'SF'),1,0)

## If the rightmost letter of position or position_alt is PF, then the person qualifies for being a power forward
my_fantasy_fuel_data['PF?']  = np.where((my_fantasy_fuel_data['position'] == 'PF') | (my_fantasy_fuel_data['position_alt'] == 'PF'),1,0)

## If the rightmost letter of position or position_alt is C, then the person qualifies for being a Center
my_fantasy_fuel_data['C?']  = np.where((my_fantasy_fuel_data['position'] == 'C') | (my_fantasy_fuel_data['position_alt'] == 'C'),1,0)

## Adding a column to reflect the full name of the player as a concatenation of first_name, a space, and last_name
my_fantasy_fuel_data['Name'] = my_fantasy_fuel_data['first_name'] + ' ' + my_fantasy_fuel_data['last_name']

## Printing the data to check if what we wanted happened or not
my_fantasy_fuel_data

,first_name,last_name,position,position_alt,injury_status,team,opp,salary,value_projection,ppg_actual,Guard?,Forward?,UTIL?,PG?,SG?,SF?,PF?,C?,Name
0,Luka,Doncic,PG,SF,NaN,DAL,LAC,11000,53.8,4.89,1,1,1,1,0,1,0,0,Luka Doncic
1,James,Harden,PG,SG,NaN,BKN,BOS,10300,49.0,4.76,1,0,1,1,1,0,0,0,James Harden
2,Kevin,Durant,SF,PF,NaN,BKN,BOS,9600,46.5,4.84,0,1,1,0,0,1,1,0,Kevin Durant
3,Julius,Randle,PF,C,NaN,NY,ATL,9700,46.3,4.77,0,1,1,0,0,0,1,1,Julius Randle
4,Jayson,Tatum,PF,NaN,NaN,BOS,BKN,9400,45.0,4.79,0,1,1,0,0,0,1,0,Jayson Tatum
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Brandon,Goodwin,SG,NaN,O,ATL,NY,3000,0.0,0.00,1,0,1,0,1,0,0,0,Brandon Goodwin
73,Jaylen,Brown,SG,NaN,O,BOS,BKN,3000,0.0,0.00,1,0,1,0,1,0,0,0,Jaylen Brown
74,JJ,Redick,SG,NaN,O,DAL,LAC,3000,0.0,0.00,1,0,1,0,1,0,0,0,JJ Redick
75,Spencer,Dinwiddie,SG,NaN,O,BKN,BOS,3000,0.0,0.00,1,0,1,0,1,0,0,0,Spencer Dinwiddie


### Game Data
To select number platers from atleast 2 games
This will need to be run every day based on the number of games and teams playing

In [7]:
my_fantasy_fuel_data['Game'] = np.where((my_fantasy_fuel_data['team'] == 'DAL') | (my_fantasy_fuel_data['opp'] == 'DAL'),'DAL-LAC',
                                        np.where((my_fantasy_fuel_data['team'] == 'NY') | (my_fantasy_fuel_data['opp'] == 'NY'),'NY-ATL',
                                                 np.where((my_fantasy_fuel_data['team'] == 'BOS') | (my_fantasy_fuel_data['opp'] == 'BOS'),'BOS-BKN',0)))

### Data for the model

*Not all the variables declared here have been used in the model*

In [8]:
## Variable to reflect number of games from which players must be selected from
min_games_selected = 2

## Variable to reflect the total available salary 
total_salary_available = 50000

## Variable to reflect number of positions that needs to be filled
n_available_positions = 8

## Variable to reflect number of players available for selection on any given day
n_available_players = len(my_fantasy_fuel_data)

## Variable refleting value projections of all the players available for selection
value_projection = my_fantasy_fuel_data.loc[:,'value_projection']

## Variable reflecting salaries of each player available each day for selection
player_salary = my_fantasy_fuel_data.loc[:,'salary']

## A 2d matrix which has a '1' if a player can be selected for a position, and a 0 therwise
## This will be used in multiplication with other matrices to form constraints
selection_martix = my_fantasy_fuel_data[['PG?','SG?','SF?','PF?','C?','Guard?','Forward?','UTIL?']].to_numpy()

## A list of the names of all the players available for selection
player_name = my_fantasy_fuel_data[['Name']].to_numpy()

## A list of primary position of all the players available for selection
player_position = my_fantasy_fuel_data.loc[:,'position']

## A list of alternate position of all the players available for selection
player_position_alt = my_fantasy_fuel_data.loc[:,'position_alt']

## A list of unique games available on a particular day
games_available = my_fantasy_fuel_data['Game'].unique()

## Make Model

In [9]:
# Create a model
model = ConcreteModel()

# Variables
model.n_selected = Var( range(n_available_players), range(n_available_positions), within= Binary )

# Constraints
model.player_conlist = ConstraintList()

### Limit a player to max one position
for i in range(n_available_players):
  select_expr = 0.0
  for j in range(n_available_positions):
      select_expr += model.n_selected[i,j] 
  model.player_conlist.add( select_expr <= 1)

### Limit one player to each position
for i in range(n_available_positions):
  select_expr = 0.0
  for j in range(n_available_players):
    select_expr += model.n_selected[j,i] * selection_martix[j,i]
  model.player_conlist.add( select_expr == 1)

### Salary Limit
salary_limit_expr = 0.0
for i in range(n_available_players):
  for j in range(n_available_positions):
    salary_limit_expr += player_salary[i] * model.n_selected[i,j]
model.player_conlist.add( salary_limit_expr <= total_salary_available)

### Player to be selected from atleast 2 available games
select_expr = 0.0
model.games_conlist = ConstraintList()
for i in range(len(games_available)):
  for j in range(n_available_players):
    select_expr += model.n_selected[j,i]
    if my_fantasy_fuel_data['Game'][j] or my_fantasy_fuel_data['Game'][j] in games_available[i]:
      model.games_conlist.add( model.n_selected[j,i] <= 1)
    else:
      model.games_conlist.add( model.n_selected[j,i] == 0)
model.games_conlist.add( select_expr >= 2)


# Objective Function
# We are defining the function as the product of two arrays. One is a binary array which tells which player has been selected, and the other is the array with the value projections from Fantasy Fuel DB.
objective_expr = 0.0
for i in range(n_available_players):
  for j in range(n_available_positions):
    objective_expr += model.n_selected[i,j] * value_projection[i]

# Describing the model objective
model.total_fppg = Objective(
    expr = objective_expr,
    sense = maximize
)

# Describing the solver type
opt = SolverFactory('cbc')
opt_success = opt.solve(model, tee= False, timelimit= 60)

print(value(model.total_fppg))
for i in range(n_available_players):
  for j in range(n_available_positions):
    if value(model.n_selected[i,j]) > 0.9:
      print(player_name[i],"---->",player_position[i],'---->',player_position_alt[i],'---->',value(model.n_selected[i,:]))


244.70000000000005
['Luka Doncic'] ----> PG ----> SF ----> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
['Kawhi Leonard'] ----> SF ----> nan ----> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
['Bogdan Bogdanovic'] ----> PG ----> SG ----> [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['Tim Hardaway Jr.'] ----> SG ----> SF ----> [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
['Alec Burks'] ----> SF ----> nan ----> [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['Tristan Thompson'] ----> C ----> nan ----> [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
['Jalen Brunson'] ----> PG ----> SG ----> [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['Taj Gibson'] ----> PF ----> C ----> [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]


## Finding next best solutions

In [ ]:
first_obj_value = value(model.total_fppg)
current_obj_value = first_obj_value

model.solution_eliminator_conlist = ConstraintList()

solution_counter = 0

while solution_counter < 100:

  eliminate_single_solution_expr = 0.0
  n_vars_equal_one = 0

  for i in range(n_available_players):
    for j in range(n_available_positions):
      if(value(model.n_selected[i,j] > 0.9)):
        eliminate_single_solution_expr += (+1) * model.n_selected[i,j]
        n_vars_equal_one += 1
      else:
        eliminate_single_solution_expr += (-1) * model.n_selected[i,j]
  model.solution_eliminator_conlist.add( eliminate_single_solution_expr <= n_vars_equal_one - 1)
  model.solution_eliminator_conlist.add( model.total_fppg <= current_obj_value - 0.01)

  opt = SolverFactory('cbc')
  opt_success = opt.solve(model, tee= False, timelimit= 60)

  current_obj_value = value(model.total_fppg)
  solution_counter += 1

  print(solution_counter,"\t", current_obj_value)
  for i in range(n_available_players):
    for j in range(n_available_positions):
      if value(model.n_selected[i,j]) > 0.9:
        print(player_name[i],'---->',value(model.n_selected[i,:]))

## Force Select certain players
Based on news and hunch, if the person using the model wishes to force select certain players because they are expecting a big game from that particular player, they can enter the name of the player and run the model with the constraints added below and get a modified line up.

This can also be helpful to distinguish one's lineup from others. Since most people are trying to get the best lineup, and real world chances can lead to varied scores, forcing the model to select a particular player can help to gain an edge over the competition.

**Do not run this section if you also want to check the next 100 best solutions.**

In [11]:
# Create a model
model = ConcreteModel()

# Variables
model.n_selected = Var( range(n_available_players), range(n_available_positions), within= Binary )

# Constraints
model.player_conlist = ConstraintList()

### Limit a player to max one position
for i in range(n_available_players):
  select_expr = 0.0
  for j in range(n_available_positions):
      select_expr += model.n_selected[i,j]
  model.player_conlist.add( select_expr <= 1)

### Limit one player to each position
for i in range(n_available_positions):
  select_expr = 0.0
  for j in range(n_available_players):
    select_expr += model.n_selected[j,i] * selection_martix[j,i]
  model.player_conlist.add( select_expr == 1)

### Salary Limit
salary_limit_expr = 0.0
for i in range(n_available_players):
  for j in range(n_available_positions):
    salary_limit_expr += player_salary[i] * model.n_selected[i,j]
model.player_conlist.add( salary_limit_expr <= total_salary_available)

### Player to be selected from atleast 2 available games
select_expr = 0.0
model.games_conlist = ConstraintList()
for i in range(len(games_available)):
  for j in range(n_available_players):
    select_expr += model.n_selected[j,i]
    if my_fantasy_fuel_data['Game'][j] or my_fantasy_fuel_data['Game'][j] in games_available[i]:
      model.games_conlist.add( model.n_selected[j,i] <= 1)
    else:
      model.games_conlist.add( model.n_selected[j,i] == 0)
model.games_conlist.add( select_expr >= 2)

### Force select player

#### Enter the full name of the player to be force selected
force_select = input("Enter name: ") 

#### Enter the position for which the player should be selected
##### IF PG   enter 0
##### IF SG   enter 1
##### IF SF   enter 2
##### IF PF   enter 3
##### IF C    enter 4
##### IF G    enter 5
##### IF F    enter 6
##### IF UTIL enter 7

position_select = int(input("Enter Position:  "))

player_index = my_fantasy_fuel_data[my_fantasy_fuel_data['Name'] == force_select].index.values

for i in range(n_available_players):
  for j in range(n_available_positions):
    select_expr = 0.0
    if i == player_index and j == position_select :
      select_expr += model.n_selected[i,j]
      model.player_conlist.add( select_expr == 1)

# Objective Function
# We are defining the function as the product of two arrays. One is a binary array which tells which player has been selected, and the other is the array with the value projections from Fantasy Fuel DB.
objective_expr = 0.0
for i in range(n_available_players):
  for j in range(n_available_positions):
    objective_expr += model.n_selected[i,j] * value_projection[i]

# Describing the model objective
model.total_fppg = Objective(
    expr = objective_expr,
    sense = maximize
)

# Describing the solver type
opt = SolverFactory('cbc')
opt_success = opt.solve(model, tee= False, timelimit= 60)

print(value(model.total_fppg))
for i in range(n_available_players):
  for j in range(n_available_positions):
    if value(model.n_selected[i,j]) > 0.9:
      print(player_name[i],"---->",player_position[i],'---->',player_position_alt[i],'---->',value(model.n_selected[i,:]))

Enter name: Kevin Durant
Enter Position:  4
244.20000000000005
['Kevin Durant'] ----> SF ----> PF ----> [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
['Kawhi Leonard'] ----> SF ----> nan ----> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
['Bogdan Bogdanovic'] ----> PG ----> SG ----> [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['Alec Burks'] ----> SF ----> nan ----> [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['Rajon Rondo'] ----> PG ----> nan ----> [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
['Kevin Huerter'] ----> SG ----> SF ----> [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['Nicolas Batum'] ----> SF ----> PF ----> [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
['Taj Gibson'] ----> PF ----> C ----> [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
['Nicolas Claxton'] ----> C ----> nan ----> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]


## Force Remove Player (Avoiding the Kardashian Curse)

Based on news, hunch, and general playoff performance of a player, we might want to remove certain individuals from being selected specially if our initial optimization model is selecting them based on Fantasy Fuel Projections.

This can also be helpful in avoiding any particular bad matchups that our data does not know of due to limited data availability.

**Not applicable when looking at 100 next best solutions.**

In [ ]:
# Create a model
model = ConcreteModel()

# Variables
model.n_selected = Var( range(n_available_players), range(n_available_positions), within= Binary )

# Constraints
model.player_conlist = ConstraintList()

### Limit a player to max one position
for i in range(n_available_players):
  select_expr = 0.0
  for j in range(n_available_positions):
      select_expr += model.n_selected[i,j]
  model.player_conlist.add( select_expr <= 1)

### Limit one player to each position
for i in range(n_available_positions):
  select_expr = 0.0
  for j in range(n_available_players):
    select_expr += model.n_selected[j,i] * selection_martix[j,i]
  model.player_conlist.add( select_expr == 1)

### Salary Limit
salary_limit_expr = 0.0
for i in range(n_available_players):
  for j in range(n_available_positions):
    salary_limit_expr += player_salary[i] * model.n_selected[i,j]
model.player_conlist.add( salary_limit_expr <= total_salary_available)

### Player to be selected from atleast 2 available games
select_expr = 0.0
model.games_conlist = ConstraintList()
for i in range(len(games_available)):
  for j in range(n_available_players):
    select_expr += model.n_selected[j,i]
    if my_fantasy_fuel_data['Game'][j] or my_fantasy_fuel_data['Game'][j] in games_available[i]:
      model.games_conlist.add( model.n_selected[j,i] <= 1)
    else:
      model.games_conlist.add( model.n_selected[j,i] == 0)
model.games_conlist.add( select_expr >= 2)

### Force remove player

#### Enter the full name of the player to be force removed
force_reject = input("Enter name: ") 

player_index = my_fantasy_fuel_data[my_fantasy_fuel_data['Name'] == force_reject].index.values

for i in range(n_available_players):
  for j in range(n_available_positions):
    select_expr = 0.0
    if i == player_index:
      select_expr += model.n_selected[i,j]
      model.player_conlist.add( select_expr == 0)

# Objective Function
# We are defining the function as the product of two arrays. One is a binary array which tells which player has been selected, and the other is the array with the value projections from Fantasy Fuel DB.
objective_expr = 0.0
for i in range(n_available_players):
  for j in range(n_available_positions):
    objective_expr += model.n_selected[i,j] * value_projection[i]

# Describing the model objective
model.total_fppg = Objective(
    expr = objective_expr,
    sense = maximize
)

# Describing the solver type
opt = SolverFactory('cbc')
opt_success = opt.solve(model, tee= False, timelimit= 60)

print(value(model.total_fppg))
for i in range(n_available_players):
  for j in range(n_available_positions):
    if value(model.n_selected[i,j]) > 0.9:
      print(player_name[i],"---->",player_position[i],'---->',player_position_alt[i],'---->',value(model.n_selected[i,:]))

Enter name: Tristan Thompson
244.60000000000002
['Luka Doncic'] ----> PG ----> SF ----> [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['Kawhi Leonard'] ----> SF ----> nan ----> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
['Kemba Walker'] ----> PG ----> nan ----> [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
['Bogdan Bogdanovic'] ----> PG ----> SG ----> [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['Alec Burks'] ----> SF ----> nan ----> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
['Jalen Brunson'] ----> PG ----> SG ----> [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['Taj Gibson'] ----> PF ----> C ----> [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
['Nicolas Claxton'] ----> C ----> nan ----> [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]


## Taking Players with maximum variance

As a measure to differentiate the team, and ride on the luck more, we have decided to go with the players with higher variance in their performance. We hope they outplay their competition on D-Day.

#### Set up additional data

This will be done using the NBA Fantasy Fuel Data uptil April.

Standard deviation of each player's fantasy points per game will be considered as the measure of variance.

Then, the standard deviation column will be normalized.

A new array of length equal to available players for that day will contain the normalized values of players available for selection that day.

In [ ]:
## Loading the fantasy fuel NBA data till April
nba_df = pd.read_excel("/content/drive/MyDrive/OPIM 5641 w David/Project/NBA_Data.xlsx")

## Creating a df from the table with player, and the mean, median, and var of their fantasy points
nba_summ = nba_df.groupby('Name').agg({'FantasyPointsDraftKings':'std'})
nba_summ.reset_index()

## Normalizing the DraftsKings Point column
nba_summ['Scaled'] = nba_summ['FantasyPointsDraftKings']/nba_summ['FantasyPointsDraftKings'].abs().max()

## Create the needed array
my_std_array = []

for i in range(n_available_players):
  for j in range(len(nba_summ)):
    if my_fantasy_fuel_data['Name'][i] == nba_summ.reset_index()['Name'][j]:
      my_std_array.append(nba_max_scaled['FantasyPointsDraftKings'][j])
  my_std_array.append(0)

## Convert it to a pandas series
my_std_array_series = pd.Series(my_std_array)

## Make a new value projection column to optimize
value_projection_new = np.zeros(n_available_players)
value_projection_new_series = pd.Series(value_projection_new)
for i in range(n_available_players):
  value_projection_new[i] = my_std_array_series[i] * value_projection[i]

#### The same model as before, with a few changes to reflect the effect of standard deviation

In [ ]:
# Create a model
model = ConcreteModel()

# Variables
model.n_selected = Var( range(n_available_players), range(n_available_positions), within= Binary )

# Constraints
model.player_conlist = ConstraintList()

### Limit a player to max one position
for i in range(n_available_players):
  select_expr = 0.0
  for j in range(n_available_positions):
      select_expr += model.n_selected[i,j] 
  model.player_conlist.add( select_expr <= 1)

### Limit one player to each position
for i in range(n_available_positions):
  select_expr = 0.0
  for j in range(n_available_players):
    select_expr += model.n_selected[j,i] * selection_martix[j,i]
  model.player_conlist.add( select_expr == 1)

### Salary Limit
salary_limit_expr = 0.0
for i in range(n_available_players):
  for j in range(n_available_positions):
    salary_limit_expr += player_salary[i] * model.n_selected[i,j]
model.player_conlist.add( salary_limit_expr <= total_salary_available)

### Player to be selected from atleast 2 available games
select_expr = 0.0
model.games_conlist = ConstraintList()
for i in range(len(games_available)):
  for j in range(n_available_players):
    select_expr += model.n_selected[j,i]
    if my_fantasy_fuel_data['Game'][j] or my_fantasy_fuel_data['Game'][j] in games_available[i]:
      model.games_conlist.add( model.n_selected[j,i] <= 1)
    else:
      model.games_conlist.add( model.n_selected[j,i] == 0)
model.games_conlist.add( select_expr >= 2)

# Objective Function
# We are defining the function as the product of two arrays. One is a binary array which tells which player has been selected, and the other is the array with the value projections from Fantasy Fuel DB.
objective_expr = 0.0
total_actual_fppg = 0.0
for i in range(n_available_players):
  for j in range(n_available_positions):
    objective_expr += model.n_selected[i,j] * value_projection_new[i]
    total_actual_fppg += model.n_selected[i,j] * value_projection[i]

# Describing the model objective
model.total_fppg = Objective(
    expr = objective_expr,
    sense = maximize
)

# Describing the solver type
opt = SolverFactory('cbc')
opt_success = opt.solve(model, tee= False, timelimit= 60)

print("Total Modified Fantasy Points Projection:  ", value(model.total_fppg))
print("Actual Fantasy Points Projection:          ", value(total_actual_fppg))
for i in range(n_available_players):
  for j in range(n_available_positions):
    if value(model.n_selected[i,j]) > 0.9:
      print(player_name[i],"---->",player_position[i],'---->',player_position_alt[i],'---->',value(model.n_selected[i,:]))

Total Modified Fantasy Points Projection:   182.98005372247698
Actual Fantasy Points Projection:           235.70000000000005
['Kevin Durant'] ----> SF ----> PF ----> [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['Jayson Tatum'] ----> PF ----> nan ----> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]
['Kristaps Porzingis'] ----> PF ----> C ----> [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
['Evan Fournier'] ----> SG ----> SF ----> [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]
['Joe Harris'] ----> SG ----> SF ----> [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
['Tristan Thompson'] ----> C ----> nan ----> [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
['Ivica Zubac'] ----> C ----> nan ----> [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
['Mike James'] ----> PG ----> nan ----> [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


## Final Team Selection Criteria

We looked at all the teams provided by the various models we had. After comparing individuals, doing research through news, and other online platforms, we decided to force add Kristaphs Porzingis at the UTIL position to the lineup suggested by our first model and then run an optimization for it.

The resulting Team is:

Position | Player Name 
-------------------|------------------
PG       | B. Bogdanovic 
SG       | L. Shamet
SF       | A. Burks
PF       | N. Batum
C        | T. Gibson
G        | L. Doncic
F        | K. Leonard
UTIL     | K. Porzingis 